In [1]:
import json
import pandas as pd
import math
import os
import xmltodict
import traceback
from slpp import slpp as lua
from compress_lua_table import CompressLuaTable as compress

In [2]:
class bcolors:
    OK = '\033[92m' #GREEN
    WARNING = '\033[93m' #YELLOW
    FAIL = '\033[91m' #RED
    RESET = '\033[0m' #RESET COLOR

In [6]:
types = {}
_types = {}
heads = set()
# with open('SkillEffectTable.json', 'rb') as f:
#     data = json.load(f)
#     for field in data['Fields']:
#         types[field['FieldName']] = field
#         _types[field['FieldName']] = field['FieldTypeName']
#         heads.add(field['FieldName'])

# set(_types.values())
dir = 'E:\WorkSpace\ROGame-dev\config\Table\Configs'
for file in os.listdir(dir):
    with open(os.path.join(dir, file), 'rb') as file:
        data = json.load(file)
        for item in data['TableLocations']:
            print(item['ExcelPath'])

AccessoryTable.csv
AchievementBadgeTable.csv
AchievementDetailTable.csv
AchievementIndexTable.csv
AchievementPointsAwardTable.csv
ActionTable.csv
ActivityComposeTable.csv
ActivityExtraAwardTable.csv
ActivitySceneTable.csv
ActivityTimeScheduleDiceTable.csv
AffixTable.csv
AlchemistMake.csv
AnimationTable.csv
AttraddMatchTable.csv
AttraddRecomTable.csv
AttrDecision.csv
AttributeAttrLimit.csv
AttrInfoTable.csv
AttrPointBaseLvTable.csv
AttrPointNeed.csv
AuctionIndexTable.csv
AuctionTable.csv
AudioCommonTable.csv
AudioStoryTable.csv
AutoAddSkillPointTable.csv
AutoAddSkilPointDetailTable.csv
AwardPackTable.csv
AwardTable.csv
BagExpandTable.csv
BarberStyleTable.csv
BarberTable.csv
BarrowTable.csv
BaseLvTable.csv
BasePackageTable.csv
BattleCardGlobal.csv
BattleCardLv.csv
BattleCardTask.csv
BattleCardTaskInterface.csv
BattleGroundLvRangeTable.csv
BattleVehicleTable.csv
BGMHouseTable.csv
BigPicTipTable.csv
BlackBoxCalcTable.csv
BlackBoxRecordTable.csv
BlackBoxValueTable.csv
BlackCurtainContentTab

In [ ]:
data = pd.read_csv('SkillEffectTable.csv').drop([0])
columns = list(set(data.columns.tolist()) - heads)
data = data.drop(columns=columns)
lua_raw_data = data.to_dict('index')

def check_default(_v, cast, default):
    def is_nan(__v):
        if type(__v) == float:
            return math.isnan(__v)
    return default if is_nan(_v) else cast(_v)


def sequence_to_dict(value, partten, length, cast):
    if value is None or value.strip() == '':
        return value
    array = {}
    sequence = value.split(partten)
    if len(sequence) == length:
        for i in range(length):
            array[i + 1] = cast(sequence[i])
        return array
    return value


def vector_to_list(value, partten, cast, func=None, *args):
    if value is None or value.strip() == '':
        return value
    l = []
    sequence = value.split(partten)
    for s in sequence:
        if func is not None:
            l.append(func(s, *args))
        elif s.strip() != '':
            l.append(cast(s))
    return l


def iteritems_recursive(d):
    t = {}
    for k, v in d.items():
        if isinstance(v, dict):
            t[k] = iteritems_recursive(v)
        else:
            _type = _types[k]
            default = types[k]['DefaultValue']
            if _type == 'string':
                t[k] = check_default(v, str, default)
            elif _type == 'int':
                t[k] = check_default(v, int, default)
            elif _type == 'float':
                t[k] = check_default(v, float, default)
            elif _type == 'bool':
                t[k] = 'false' if v == 0 or v == 'nan' else 'true'
            elif _type == 'Sequence<int, 2>':
                t[k] = sequence_to_dict(check_default(v, str, default), '=', 2, int)
            elif _type == 'vector<Sequence<int, 2>>':
                args = ['=', 2, int]
                t[k] = vector_to_list(check_default(v, str, default), '|', int, sequence_to_dict, *args)
            elif _type == 'vector<Sequence<int, 3>>':
                args = ['=', 3, int]
                t[k] = vector_to_list(check_default(v, str, default), '|', int, sequence_to_dict, *args)
            elif _type == 'vector<int>':
                t[k] = vector_to_list(check_default(v, str, default), '|', int)
            elif _type == 'vector<float>':
                t[k] = vector_to_list(check_default(v, str, default), '|', float)
            elif _type == 'vector<string>':
                t[k] = vector_to_list(check_default(v, str, default), '|', str)
            elif _type == 'vector<vector<int>>':
                args = ['=', int]
                t[k] = vector_to_list(check_default(v, str, default), '|', int, vector_to_list, *args)
            else:
                print(f'`{bcolors.FAIL}' + _type + f'{bcolors.RESET}` is not processed!')
                # return
    return t

table = iteritems_recursive(lua_raw_data)
with open('SkillTable.lua', 'w', encoding='utf-8') as f:
    f.write('return ' + lua.encode(table))

In [18]:
def is_int(n):
    try:
        int(n)
    except ValueError:
        return False
    return True

def is_float(n):
    try:
        float(n)
    except ValueError:
        return False
    return True

def is_boolean(n):
    if n == 'true' or n == 'false':
        return True
    return False

def to_boolean(n):
    if n == 'true': return True
    elif n == 'false': return False

def filter_key(k):
    return k == '@xmlns:xsd' or k == '@xmlns:xsi'

def iter_xml_recursive(d):
    t = {}
    for k, v in d.items():
        if isinstance(v, dict):
            t[k] = iter_xml_recursive(v)
        elif isinstance(v, list):
            t[k] = iter_xml_recursive({i : v[i] for i in range(len(v))})
        else:
            if v is None:
                t[k] = None
            elif is_int(v):
                t[k] = int(v)
            elif is_float(v):
                t[k] = float(v)
            elif is_boolean(v):
                t[k] = to_boolean(v)
            elif not filter_key(k):
                t[k] = v
                # return
    return t

with open('E:\WorkSpace\ROGame-dev\config\Assets\Resources\SkillData\Activity_XieNengRuQin\Monster_ShenGuan_ShengGuangFaZhen.txt', 'rb') as f:
    table = iter_xml_recursive(dict(xmltodict.parse(f)))
    # print(json.loads(json.dumps(xmltodict.parse(f))))

In [17]:
from compress_lua_table import CompressLuaTable as compress
compress.process_file(table, 'Monster_ShenGuan_ShengGuangFaZhen')

In [ ]:
dir = 'E:\WorkSpace\ROGame-dev\config\Assets\Resources\SkillData'
EDITOR_DIR = './SkillData/'
if not os.path.exists(EDITOR_DIR):
    os.mkdir(EDITOR_DIR)
for path, dirs, fs in os.walk(dir):
    for f in fs:
        if f.endswith('.txt'):
            # print(f'processing {bcolors.OK}' + os.path.join(path, f) + f'{bcolors.RESET}...')
            # print(os.path.join(EDITOR_DIR + path[len(dir) + 1:], f.replace('.txt', '.lua')))
            _dir = os.path.join(EDITOR_DIR + path[len(dir) + 1:])
            if not os.path.exists(_dir):
                os.mkdir(_dir)
            with open(os.path.join(path, f), 'rb') as file:
                try:
                    table = iter_xml_recursive(dict(xmltodict.parse(file)))
                    with open(os.path.join(_dir, f.replace('.txt', '.lua')), 'w', encoding='utf-8') as w:
                        w.write('local ' + f[:-4] + ' = ' + lua.encode(table) + '\n return ' + f[:-4])
                except Exception as e:
                    print(f'{bcolors.FAIL} error: ' + os.path.join(path, f) + f'{bcolors.RESET}')
                    traceback.print_exc()


 error: E:\WorkSpace\ROGame-dev\config\Assets\Resources\SkillData\Player_Paladin\Player_Paladin_XiSheng.txt
 error: E:\WorkSpace\ROGame-dev\config\Assets\Resources\SkillData\Player_Paladin\Mount\Player_Paladin_XiSheng.txt


Traceback (most recent call last):
  File "C:\Users\MINGZH~1\AppData\Local\Temp/ipykernel_19136/1264675098.py", line 16, in <module>
    table = iter_xml_recursive(dict(xmltodict.parse(file)))
  File "D:\Python\Python38\lib\site-packages\xmltodict.py", line 325, in parse
    parser.ParseFile(xml_input)
xml.parsers.expat.ExpatError: unclosed token: line 113, column 8
Traceback (most recent call last):
  File "C:\Users\MINGZH~1\AppData\Local\Temp/ipykernel_19136/1264675098.py", line 16, in <module>
    table = iter_xml_recursive(dict(xmltodict.parse(file)))
  File "D:\Python\Python38\lib\site-packages\xmltodict.py", line 325, in parse
    parser.ParseFile(xml_input)
xml.parsers.expat.ExpatError: no element found: line 111, column 42


 error: E:\WorkSpace\ROGame-dev\config\Assets\Resources\SkillData\Player_Paladin_F\Mount\Player_Paladin_XiSheng.txt


Traceback (most recent call last):
  File "C:\Users\MINGZH~1\AppData\Local\Temp/ipykernel_19136/1264675098.py", line 16, in <module>
    table = iter_xml_recursive(dict(xmltodict.parse(file)))
  File "D:\Python\Python38\lib\site-packages\xmltodict.py", line 325, in parse
    parser.ParseFile(xml_input)
xml.parsers.expat.ExpatError: no element found: line 111, column 42


In [1]:
import re

str = 'vector<string>'
str2 = 'vector<Sequence<float, 4>>'
pattern = re.compile('([1-9]\d*|string)')
type_pattern = 'Sequence|vector'
print(pattern.findall(str))
print(pattern.match(str2))
print(re.match('vector<[a-z]*>', str))
print(re.match('vector<(Sequence|vector)<[a-z]*, [1-9]\d*>>$', str2))
print(os.cpu_count())

['string']
None
<re.Match object; span=(0, 14), match='vector<string>'>
<re.Match object; span=(0, 26), match='vector<Sequence<float, 4>>'>
16


In [3]:
%run main.py -k --f client --csv

processing AccessoryTable.csv ...
processing AchievementBadgeTable.csv ...
processing AchievementDetailTable.csv ...
processing AchievementIndexTable.csv ...
processing AchievementPointsAwardTable.csv ...
processing ActionTable.csv ...
processing ActivityComposeTable.csv ...
processing ActivityExtraAwardTable.csv ...
processing ActivitySceneTable.csv ...
processing ActivityTimeScheduleDiceTable.csv ...
processing AffixTable.csv ...
processing AlchemistMake.csv ...
processing AnimationTable.csv ...
processing AttraddMatchTable.csv ...
processing AttraddRecomTable.csv ...
processing AttrDecision.csv ...
processing AttributeAttrLimit.csv ...
processing AttrInfoTable.csv ...
processing AttrPointBaseLvTable.csv ...
processing AttrPointNeed.csv ...
processing AuctionIndexTable.csv ...
processing AuctionTable.csv ...
processing AudioCommonTable.csv ...
processing AudioStoryTable.csv ...
processing AutoAddSkillPointTable.csv ...
processing AutoAddSkilPointDetailTable.csv ...
processing AwardPa

In [2]:
import subprocess

_dir = 'Table-client/'
for file in os.listdir(_dir):
    func = 'optimizer(\"{0}\", \"{1}\")'.format(_dir + file[:-4], file[:-4])
    try:
        result = subprocess.check_output(['lua53', '-l', 'DataTableOptimizer', '-e', func])
    except Exception as e :
        print(file)

In [2]:
import subprocess
import traceback


for file in os.listdir('./Table-client'):
    func = '_load(\"{0}\")'.format('Table-client.' + file[:-4])
    try:
        result = subprocess.check_output(['lua53', '-l', 'test', '-e', func])
    except Exception as e :
        print(file)
        # pass
        # traceback.print_exc()

In [33]:
import re
import sys
from numbers import Number
from slpp import slpp as lua
import six
import json


tab = ''
newline = ''
depth = 0

def __encode(obj):
    s = ''
    global depth
    global newline
    global tab
    if isinstance(obj, str):
        s += '"%s"' % obj.replace(r'"', r'\"')
    elif six.PY2 and isinstance(obj, unicode):
        s += '"%s"' % obj.encode('utf-8').replace(r'"', r'\"')
    elif six.PY3 and isinstance(obj, bytes):
        s += '"{}"'.format(''.join(r'\x{:02x}'.format(c) for c in obj))
    elif isinstance(obj, bool):
        s += str(obj).lower()
    elif obj is None:
        # pass
        s += 'nil'
    elif isinstance(obj, Number):
        s += str(obj)
    elif isinstance(obj, dict):
        depth += 1
        s += "{"
        key_list = ['%s' for k in obj.keys()]
        contents = [(key + '%s') % (k, __encode(v)) for (k, v), key in zip(obj.items(), key_list)]
        s += (',%s' % newline).join(contents)
        depth -= 1
        s += "}"
    return s

def encode(obj):
    s = ''
    if isinstance(obj, str):
        s += '"%s"' % obj.replace(r'"', r'\"')
    elif isinstance(obj, bool):
        s += str(obj).lower()
    elif obj is None:
        s += 'nil'
    elif isinstance(obj, Number):
        s += str(obj)
    elif isinstance(obj, dict):
        s += "{"
        contents = [encode(v) for _, v in obj.items()]
        s += ','.join(contents)
        s += "}"
    return s


def compress_lua(obj, name):
    s = 'local {} = {\n'.format(name)
    for _, value in obj.items():
        line = '\t{'
        for k, items in value.items():
            line += encode(items) + ','
        line = line[:-1] + '},\n'
        s += line

    # define default table
    s += '}\n\nlocal __default_table = {'
    for key, _ in indices.items():
        s += key + ','
    s = s[:-1] + '}\n'

    # add postfix

    return s

    
with open('./Table-client/AwardPackTable.lua', 'r') as f:
    table_name = 'AwardPackTable'
    text = f.read()
    obj = lua.decode(text)
    res = compress_lua(obj)
    res = 'local {} = '.format(table_name) + res
    # res = __encode(obj)
    with open('test.lua', 'w', encoding='utf-8') as w:
        w.write(res)
        w.write("\ndo\n")
        w.write("\tlocal base = {__index = __default_table, __newindex = function() error(\"Attempt to modify read-only table\") end}\n")
        w.write("\tfor k, v in pairs(%s) do\n" % (table_name))
        w.write("\t\tsetmetatable(v, base)\n")
        w.write("\tend\n")
        w.write("\tbase.__metatable = false\n")
        w.write("end\n")
        w.write("\nreturn %s\n" % (table_name))
